In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tasks import NER
from utils_ner import TokenClassificationDataset
import torch
import  nltk

In [8]:
# load phobert-ner đã được fine-tuned
phobert_ner = AutoModelForTokenClassification.from_pretrained("huyenbui117/Covid19_phoNER")
# load tokenizer của phobert
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
labels = NER().get_labels(None)
labels

['O',
 'B-ORGANIZATION',
 'O-ORGANIZATION',
 'I-ORGANIZATION',
 'B-LOCATION',
 'I-LOCATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-DATE',
 'I-DATE',
 'B-PATIENT_ID',
 'I-PATIENT_ID',
 'B-AGE',
 'I-AGE',
 'B-NAME',
 'I-NAME',
 'B-JOB',
 'I-JOB',
 'B-TRANSPORTATION',
 'I-TRANSPORTATION',
 'B-GENDER']

In [31]:
sentences = ["Bệnh nhân BN-946401 là NKH, thường trú tại thôn Thành Trung, xã Phồn Xương, huyện Yên Thế, tỉnh Bắc Giang",
            "thực hiện test nhanh COVID-19 tại Trung tâm Y tế huyện Hữu Lũng cho kết quả nghi ngờ ngày 03/11/2021.",
            "Ngày 04/11/2021 có kết quả xét nghiệm dương tính với SARS-CoV2 của Bệnh viện Đa khoa tỉnh Lạng Sơn."]

In [39]:
for sentence in sentences:
    
    input_ids = torch.tensor([tokenizer.encode(sentence, "utf-8")])
    tokens = nltk.word_tokenize(sentence)
    print(tokens)
    outputs = phobert_ner(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)
    print(predictions)
    for i in [(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
        print(i)
    print('---------------------------------------------')


['Bệnh', 'nhân', 'BN-946401', 'là', 'NKH', ',', 'thường', 'trú', 'tại', 'thôn', 'Thành', 'Trung', ',', 'xã', 'Phồn', 'Xương', ',', 'huyện', 'Yên', 'Thế', ',', 'tỉnh', 'Bắc', 'Giang']
tensor([[ 0,  0,  0, 18, 19, 19, 19,  0,  0,  0,  0,  0,  0,  0,  0,  4,  5,  5,
          5,  0,  4,  5,  5,  5,  5,  0,  4,  5,  5,  0,  4,  5,  5,  0,  0,  0,
          0,  0,  0]])
('Bệnh', 'O')
('nhân', 'O')
('BN-946401', 'O')
('là', 'B-TRANSPORTATION')
('NKH', 'I-TRANSPORTATION')
(',', 'I-TRANSPORTATION')
('thường', 'I-TRANSPORTATION')
('trú', 'O')
('tại', 'O')
('thôn', 'O')
('Thành', 'O')
('Trung', 'O')
(',', 'O')
('xã', 'O')
('Phồn', 'O')
('Xương', 'B-LOCATION')
(',', 'I-LOCATION')
('huyện', 'I-LOCATION')
('Yên', 'I-LOCATION')
('Thế', 'O')
(',', 'B-LOCATION')
('tỉnh', 'I-LOCATION')
('Bắc', 'I-LOCATION')
('Giang', 'I-LOCATION')
---------------------------------------------
['thực', 'hiện', 'test', 'nhanh', 'COVID-19', 'tại', 'Trung', 'tâm', 'Y', 'tế', 'huyện', 'Hữu', 'Lũng', 'cho', 'kết', 'quả', 'ng